## 进阶知识与对抗模型实现

为了设计一个智能自动对话系统，重要的是让聊天机器人在对话互动中变得有同理心。但先目前建立共情对话系统仍然存在障碍：

- 情感提示嵌入仅仅只使用了粗粒度(对话级)的情感信息（情感词典）。
- 在对话生成过程中，仍然难以准确捕捉人类语句种情感的细微差别。
- 仅仅依赖对话历史，而忽视用户反馈对生成的回复的潜在影响，进一步加剧了上述缺陷，从而导致不适当回复。

![基准数据集共情对话](./images/04-1.png)

图中是基准数据集共情对话的示例：

Utterence1（即“new”、“job”）和 Utterence2（即“amazing”、“excited”）中的情感词具有细粒度的情感联系。用户的反馈（Utterence3）也与目标回复（Utterence2）有着密切的情感联系。因此，明确建模细粒度的情绪因素并考虑用户反馈是必要的。

基于此，通过实现一个多尺度的对抗性共情对话生成模型，通过生成更合适的共情回复来应对上述挑战。

该模型的设计思路如下：

1. **多分辨率情感建模**：联合考虑粗粒度对话级情绪和细粒度 token 级情绪来生成回复
2. **交互式对抗训练**：利用用户反馈设计双判别器架构
3. **WGAN-GP稳定训练**：相比传统 GAN 在文本生成中的训练更稳定

### 1. 任务定义

对话中同时存在语义语境和情感语境，语义语境 $U$ 指的是话语的顺序，即 $U=[U_1,…,U_M]$ 。将 $U$ 展平为一个长的 $token$ 序列，并在句的开头插入一个 $CLS$ 标记，即 $U=[CLS,x_1,…,x_m]$ 。情感语境 $E$ 考虑具有不同粒度的情感，即 $E=[LAB,w_1,…,w_e]$ ，其中 $w_i$ 是语义语境 $U$ 中的情感词，$LAB$ 是一种特殊的情感标记，用于推导对话语境的情感状态。给定 $U$ 和 $E$，模型目标旨在通过最大化概率 $P(Y | U,E)=\prod_{n=1}^{N} P(y_n | y_{<n},U,E)$ 来生成 $n$ 长度回复 $Y=\{y_1,…,y_n\}$ 。

EmpDG 模型由两个主要组成部分：共情器和交互判别器。框架基于 WGAN 进行设计，generator 是基于 transformer 设计的 encoder-decoder 架构。discriminator 是使用的 Wasserstein-1 distance。架构图如下：

![EmpDG 模型结构](./images/04-2.png)

在编码过程中，对语义语境和多尺度情感语境进行编码；而解码器则将语义语境和情感语境融合在一起，以产生回复。鉴别器包含两个部分，即语义鉴别器和情感鉴别器，在训练过程中，两个鉴别器还与用户反馈（下一个话语和下一个话语的相应情感词）进行交互。通过最小化 **Wasserstein-1距离** 来优化鉴别器，使用分类结果的总和作为训练信号来增强回复生成器情感感知能力。

## 2. 对抗训练概述

### 2.1 传统对话生成的核心瓶颈

在共情对话（Empathetic Dialogue）任务中，传统的 **Seq2Seq** 或 **Transformer** 生成模型常通过最大似然估计（MLE）进行训练，这带来了三个长期存在的问题：

1. **暴露偏差（Exposure Bias）**

   * 训练阶段：模型总是接收到来自人工标注的“正确”上一句话作为输入。
   * 推理阶段：输入来自模型自身的预测，一旦出错，后续生成会被逐步放大，可能彻底偏离合理的对话轨迹。

2. **通用响应问题（Generic Response Problem）**

   * MLE 倾向于最大化高频响应的概率，因此模型常输出 **安全但无信息量** 的句子（如：“我不知道”“是的”“哦”）。
   * 这些“保险”回答虽然减少了语法错误，但降低了对话的多样性与趣味性。

3. **情感一致性缺失（Lack of Emotional Alignment）**

   * 模型可能忽略用户的情绪状态。例如，用户表达悲伤时，模型可能返回欢快、无关痛痒的回复，造成情感错位。
   * 在共情场景中，这种失配尤其致命，会显著降低用户感知到的情感支持。

### 2.2 对抗训练的引入与作用

**对抗训练（Adversarial Training）** 在 EmpDG 中的核心思想：

* **生成器 G**：尝试生成与人类响应同样自然、情感匹配且上下文一致的回复，以“欺骗”判别器。
* **判别器 D**：判定输入回复是人工生成还是模型生成，并在 EmpDG 中进一步细分为：

  * **语义判别器**（Semantic Discriminator）：评估生成回复与真实回复的语义一致性。
  * **情感判别器**（Emotional Discriminator）：评估生成回复的情绪是否与对话上下文匹配。

这种双判别器机制，不仅迫使生成器产生**多样化且信息丰富**的回复，还增强了**情感一致性**。

在训练中，判别器不断提高“识破”假回复的能力，生成器则不断优化生成质量以迷惑判别器，形成正向博弈。

### 2.3 为什么选择 WGAN（Wasserstein GAN）

#### 传统 GAN 在对话生成中的问题

1. **训练不稳定**：容易出现模式崩塌（Mode Collapse），即生成结果缺乏多样性。
2. **梯度消失**：当判别器过强时，生成器几乎收不到有用的梯度信号。
3. **损失不可解释**：原始 GAN 的损失值不能直接反映生成质量，调参和早停困难。

#### WGAN 的优势

1. **训练稳定性更高**：Wasserstein 距离（Earth Mover’s Distance）为生成器提供了连续、稳定的梯度信号。
2. **损失值可解释**：判别器输出的分数直接反映生成分布与真实分布的差距，方便监控训练进度。
3. **减少模式崩塌**：WGAN 在理论上更容易覆盖真实数据分布的多样性。
4. **结合梯度惩罚（WGAN-GP）**：避免了权重裁剪的缺陷，使模型表达能力更强。

### 2.4 WGAN 原理与在 EmpDG 中的应用

#### 2.4.1 Wasserstein 距离

Wasserstein 距离刻画了将一个概率分布“搬运”成另一个概率分布所需的最小代价：

$$
W(P_r, P_g) = \inf_{\gamma \in \Pi(P_r, P_g)} \mathbb{E}_{(x,y) \sim \gamma}[||x - y||]
$$

通过 Kantorovich-Rubinstein 对偶性可写为：

$$
W(P_r, P_g) = \sup_{||f||_L \leq 1} \mathbb{E}_{x \sim P_r}[f(x)] - \mathbb{E}_{x \sim P_g}[f(x)]
$$

其中 $||f||\_L \leq 1$ 表示 $f$ 必须是 1-Lipschitz 函数。

在 EmpDG 中：

* $P_r$：真实的（人工编写的）共情回复分布。
* $P_g$：生成器生成的共情回复分布。
* 判别器的任务是近似这个最优 1-Lipschitz 函数 $f$。

#### 2.4.2 WGAN 损失函数

与传统 GAN 不同，WGAN 移除了 Sigmoid 激活，并直接用判别器分数作为距离度量。

* **判别器（D）损失**：

$$
L_D = \mathbb{E}_{x \sim P_g}[D(x)] - \mathbb{E}_{x \sim P_r}[D(x)]
$$

* **生成器（G）损失**：

$$
L_G = -\mathbb{E}_{x \sim P_g}[D(x)]
$$

#### 2.4.3 梯度惩罚（WGAN-GP）

为了保证 1-Lipschitz 条件，WGAN-GP 使用**梯度惩罚**替代权重裁剪：

$$
L_D^{GP} = L_D + \lambda \mathbb{E}_{\hat{x} \sim P_{\hat{x}}}[(\|\nabla_{\hat{x}} D(\hat{x})\|_2 - 1)^2]
$$

其中：

* $\hat{x} = \epsilon x + (1-\epsilon)\tilde{x}$，$\epsilon \sim U(0,1)$
* $x$ 来自真实分布，\$\tilde{x}\$ 来自生成分布
* $\lambda$ 是梯度惩罚系数

#### 2.4.4 在 EmpDG 中的特化

EmpDG 的判别器不仅要区分真假，还要：

1. **语义层面**（Semantic Discriminator）：度量生成回复与目标回复的内容相似度。
2. **情感层面**（Emotional Discriminator）：评估生成回复的情绪是否与上下文情绪匹配，并借助**下一轮用户的隐式反馈**进行优化。

这种“语义 + 情感”的双判别器结构，使得 EmpDG 能在生成的多样性、上下文一致性和情感契合度上同时提升。

下面我们通过模拟生成响应和真实响应的判别器输出，来演示WGAN损失函数的计算

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

def demonstrate_wgan_loss():
    """演示WGAN损失函数的计算"""
    
    # 模拟生成响应和真实响应的判别器输出
    batch_size = 16
    
    # 判别器对生成响应的评分（越低越好，表示生成质量越差）
    neg_semantic_logits = torch.randn(batch_size, 1) * 0.5 - 1.0  # 倾向于负值
    
    # 判别器对真实响应的评分（越高越好，表示真实响应质量高）  
    pos_semantic_logits = torch.randn(batch_size, 1) * 0.5 + 1.0  # 倾向于正值
    
    # WGAN损失计算
    # 判别器损失：希望真实响应得分高，生成响应得分低
    disc_loss = torch.mean(neg_semantic_logits - pos_semantic_logits)
    
    # 生成器损失：希望生成响应得分高（欺骗判别器）
    gen_loss = torch.mean(neg_semantic_logits)
    
    print(f"判别器损失: {disc_loss.item():.4f}")
    print(f"生成器损失: {gen_loss.item():.4f}")
    print(f"真实响应平均得分: {torch.mean(pos_semantic_logits).item():.4f}")
    print(f"生成响应平均得分: {torch.mean(neg_semantic_logits).item():.4f}")
    
    return disc_loss, gen_loss

def demonstrate_gradient_penalty():
    """演示梯度惩罚的计算过程"""
    
    batch_size = 16
    hidden_size = 300
    
    # 模拟真实和生成的响应表示
    real_sample = torch.randn(batch_size, 10, hidden_size)  # (bsz, seq_len, hidden)
    fake_sample = torch.randn(batch_size, 10, hidden_size)
    
    # 创建插值样本
    alpha = torch.rand(batch_size, 1, 1)  # (bsz, 1, 1)
    interpolates = alpha * real_sample + (1 - alpha) * fake_sample
    interpolates.requires_grad_(True)
    
    # 模拟判别器对插值样本的评分
    disc_interpolates = torch.sum(interpolates, dim=(1, 2)).unsqueeze(1)  # 简化的判别器输出
    
    # 计算梯度
    gradients = torch.autograd.grad(
        outputs=disc_interpolates,
        inputs=interpolates,
        grad_outputs=torch.ones_like(disc_interpolates),
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    
    # 计算梯度范数
    slopes = torch.sqrt(torch.sum(gradients ** 2, dim=(1, 2)))
    
    # 梯度惩罚
    gradient_penalty = torch.mean((slopes - 1) ** 2)
    
    print(f"平均梯度范数: {torch.mean(slopes).item():.4f}")
    print(f"梯度惩罚: {gradient_penalty.item():.4f}")
    
    return gradient_penalty

# 运行演示
print("=== WGAN损失函数演示 ===")
demonstrate_wgan_loss()

print("\n=== 梯度惩罚演示 ===")
demonstrate_gradient_penalty()

=== WGAN损失函数演示 ===
判别器损失: -1.9422
生成器损失: -0.9883
真实响应平均得分: 0.9539
生成响应平均得分: -0.9883

=== 梯度惩罚演示 ===
平均梯度范数: 54.7723
梯度惩罚: 2891.4553


tensor(2891.4553)

### 3. 共情生成器（Empathetic Generator）

![共情生成器](./images/04-3.png)

先看左侧生成器，生成器下方是encoder，上方是decoder。encoder输入的特征有两项，分别是语义理解（semantic understanding）和多分辨率情感感知（multi-resolution emotion perception）。

**Empathetic Generator** 的核心思想是把「语义理解」与「情感感知（多分辨率）」分开用两个 Transformer 编码器表示，再把预测到的情感信号显式作为 decoder 的第一项输入（conditioning token），decoder 通过对语义与情感上下文的 cross-attention 生成带有同理心的回复。

#### 3.1 Semantic Understanding（语义理解）

**目的**：把对话的语义信息（词序列、上下文、说话者信息）编码成能被 decoder 查询的向量序列 $\mathbf{C}_u$。

**实际做法（公式对应）**：

* 输入 token 的三类 embedding 相加得到每个位置的输入向量：

  $$
  \mathbf{x}_i = e^{W}_i + e^{P}_i + e^{D}_i \in \mathbb{R}^d
  $$

  * $e^W$：word embedding
  * $e^P$：positional embedding
  * $e^D$：dialogue-state embedding（用于区分 speaker/listener、不同轮次等）
* 把 $\mathbf{x}_i$ 送入标准 Transformer 层（多头自注意力 + 残差 + LayerNorm + FFN），堆叠 $L$ 层得最终语义上下文序列：

  $$
  \widetilde{\mathbf{x}}_i^{(L)} \quad\text{组成}\quad \mathbf{C}_u = [\widetilde{\mathbf{x}}_0,\dots,\widetilde{\mathbf{x}}_m].
  $$

**设计要点**：

* 把 dialogue-state 作为 embedding 是关键：在多轮对话里，谁说的话很重要（同理心回复往往依赖听者/讲者角色）。
* 使用 Transformer 的局部/全局自注意力，能让模型聚焦上下文的关键片段（如情感触发词、事件描述）。

#### 3.2 Multi-resolution Emotion Perception（多分辨率情感感知）

**目的**：情感在对话中既有粗粒度标签（例如 dataset 给的 emotion 分类），也有细粒度的情感触发词（token-level）。单独一个 encoder 混合两类信息会削弱效果，因此用独立的 Transformer 来专门捕捉情感信号。

**实际做法（公式对应）**：

* 对情感序列 $\mathcal{E}$（包含 coarse label token + 若干被情感词典标记的情感词）做 embedding，同样三类 embedding 之和：

  $$
  \mathbf{w}_i = e^{W}_{w_i} + e^{P}_{w_i} + e^{E}_{w_i}
  $$

  其中 $e^E$ 为 emotion-state embedding（标识这是情感上下文）。
* 经过 L 层 Transformer 得到 $\widetilde{\mathbf{w}}_i$，构成情感上下文序列 $\mathbf{C}_e = [\widetilde{\mathbf{w}}_0,\dots]$。
* 用情感 CLS $\widetilde{\mathbf{w}}_0$ 与语义 CLS $\widetilde{\mathbf{x}}_0$ 拼接做情感分类器：

  $$
  \mathbf{e}_p = \mathbf{W}_e [\widetilde{\mathbf{w}}_0; \widetilde{\mathbf{x}}_0],\qquad
  P_e(e\mid\mathcal{E}) = \operatorname{softmax}(\mathbf{e}_p)
  $$

  训练时对话级情感标签用交叉熵（负对数似然）损失 $\mathcal{L}_{emo} = -\log P_e(e^\ast\mid\mathcal{E})$。

**设计要点**：

* 两个 encoder 分开参数，可以让情感 encoder 学到不同于语义 encoder 的模式（比如高频情感词、情感句法结构等）。
* 拼接语义 CLS + 情感 CLS 用来结合语义与情感的 global 信息做分类：语义 CLS 补充上下文语义线索，情感 CLS 提供情感词特征。

#### 3.3 Empathetic Response Generation（同理心回复生成）

**目的**：在 decoder 里让生成过程显式“感知”预测的情感信号，从而生成既连贯又情感一致的回复。

**实际做法（公式对应）**：

1. 把情感分类器的输出 $\mathbf{e}_p\in\mathbb{R}^{1\times q}$ 线性变换到 embedding 维度 $d$：

   $$
   \mathbf{e}_p' = \mathbf{W}_{ep}\,\mathbf{e}_p \in \mathbb{R}^{1\times d}.
   $$
2. 把 $\mathbf{e}_p'$ 作为 decoder 输入序列的第 0 个 token embedding：
   $\mathbf{E}_Y = [\mathbf{y}_0=\mathbf{e}_p',\, \mathbf{y}_1,\dots,\mathbf{y}_{j-1}]$（后者是前面已生成或 gold 的 token embeddings）。
3. Transformer decoder 层先在自注意力和层内更新后，用多头 **cross-attention**（MH-CAtt）对 **联合上下文** $\mathbf{C}=[\mathbf{C}_u;\mathbf{C}_e]$ 进行查询，得到上下文融合向量：

   $$
   \mathbf{D} = \mathbf{Y} + \mathbf{W}_m \, \text{MH-CAtt}(\mathbf{Y},\mathbf{C}),
   $$

   然后 LayerNorm、FFN 得到 $\widehat{\mathbf{Y}}$。
4. 用线性变换和 softmax 产生下一个 token 的分布：

   $$
   p(y_j \mid y_{<j},\mathbf{C}) = \operatorname{softmax}(\mathbf{W}_o \, \widehat{\mathbf{y}}^j).
   $$
5. 训练用 MLE（交叉熵）最小化 $\mathcal{L}_{gen} = -\sum_j \log p(y_j\mid y_{<j},\mathbf{C})$。

**设计要点**：

* 把情感向量放在 decoder 输入序列开头，效果类似“给 decoder 一句 prompt / conditioning token”，decoder 可以在每一步 self-attention 中把情感信息作为历史的一部分使用（既显式又简洁）。
* cross-attention 的 query 来自 decoder（带了情感信息），key/value 来自语义+情感上下文 $\mathbf{C}$，因此 decoder 能同时检索语义事实和情感相关的短语来生成回复。

### 4. 交互式鉴别器（Interactive Discriminators）

![交互式鉴别器](./images/04-4.png)

**Interactive Discriminators** 是该模型最有意思的创新点之一，因为它不只是用判别器判断真假，而是结合 **用户隐式反馈（user feedback）** 来动态地约束生成器学会“有同理心”的回应。

#### 4.1 核心思想

Interactive Discriminators 有两个判别器：

* **Semantic Discriminator** — 检查生成的回复在语义上是否贴近真实人类回复（gold response）。
* **Emotional Discriminator** — 检查生成的回复是否包含足够的情感（empathy）因素。

它们的独特之处：

1. **使用“下一轮对话”作为用户隐式反馈**：
   * 在对话中，用户说的下一句话（next utterance）本质上反映了他对你上一个回复的反应。
   * 如果用户接着展开对话，可能说明你的回复语义合理、情感到位；反之则可能缺乏共鸣。
   * 所以论文将 *下一轮的用户回复* 拆成两部分：
     * 语义部分 → 作为语义反馈（semantic feedback）
     * 情感词部分 → 作为情感反馈（emotional feedback）

2. **GAN + Wasserstein 距离**：
   * 判别器使用 Wasserstein-1 距离（WGAN）来衡量生成和真实的分布差异，并加梯度惩罚（gradient penalty）保证 1-Lipschitz。

#### 4.2 Semantic Discriminator（语义鉴别器）

输入：
* **生成回复**（Generated Response, $d_t^N$） → 负样本
* **真实回复**（Gold Response, $d_t^P$） → 正样本
* **用户语义反馈**（semantic feedback, $d_F$）
* **对话上下文向量**（context vector, $\tilde{x}_0$）

编码：
* 先用 **LSTM encoder** 把 $d_t^N$ 和 $d_t^P$ 分别编码成 hidden 表示（vector sequences）。
* 对每个样本，用多尺度的卷积核（CNN）提取 n-gram 级别的特征：
  $$
  f_t^* = \mathrm{ReLU}(d_t^* \otimes W_s + b_s)
  $$
  * 卷积核宽度不同 → 捕捉不同长度的依存关系。
  * 然后用 **max-pooling** 得到固定长度向量 $f^*$。

判别器输出：
* 把 CNN 特征 $f^*$、语义反馈向量 $d_F$、上下文向量 $\tilde{x}_0$ 相加，然后过 ReLU + 全连接层：
  $$
  D_{\mathrm{sem}}(d^*) = W_d \cdot \mathrm{ReLU}(f^* + d_F + \tilde{x}_0) + b_d
  $$
* 这里的 $D_{\mathrm{sem}}$ 输出一个标量，衡量语义贴近度。

损失函数：
* 用 **Wasserstein-1 距离** + **梯度惩罚**：
  $$
  \mathcal{L}_{d_{\mathrm{sem}}} = \frac{1}{n} \sum_t D_{\mathrm{sem}}(d_t^N) - D_{\mathrm{sem}}(d_t^P) + \beta (\| \nabla_{d'_t} D_{\mathrm{sem}}(d'_t) \|_2 - 1)^2
  $$
  * $d'_t = \alpha d_t^N + (1 - \alpha) d_t^P$ 是在正负样本之间插值得到的点。
  * 梯度惩罚项保证判别器满足 1-Lipschitz 条件。

#### 4.3 Emotional Discriminator（情绪鉴别器）

情绪鉴别器的结构与语义鉴别器几乎一样，只是：
* 输入是**情感词子序列**（emotional words sequence），而不是整句回复。
* 用户情感反馈（emotional feedback）由下一轮用户回复中的情感词构成。
* 损失函数记为 $\mathcal{L}_{d_{\mathrm{emo}}}$。

#### 4.4 训练目标整合

总的判别器损失：
$$
\mathcal{L}_d = \mathcal{L}_{d_{\mathrm{sem}}} + \mathcal{L}_{d_{\mathrm{emo}}}
$$
生成器损失（$\mathcal{L}_g$）在原有生成任务的基础上，加上判别器的负分数项：
$$
\mathcal{L}_g \; \text{ += } \; - D_{\mathrm{sem}}(d_t^N) - D_{\mathrm{emo}}(d_t^N)
$$
这样生成器会被鼓励去生成能通过两个判别器检查的回复，从而兼顾语义合理性和情感共鸣。

#### 4.5 作用与意义

* **和普通 GAN 的区别**：
  普通文本 GAN 的判别器只是判断真/假，而这里的判别器是“互动的”，因为它利用了未来对话轮次的用户隐式反馈来给生成器信号。
* **语义 + 情感双重约束**：
  一个保证内容合理（不会牛头不对马嘴），一个保证情绪到位（不会机械化冷冰冰）。
* **梯度稳定性**：
  使用 WGAN-GP 避免了原始 GAN 在文本任务中梯度消失或模式崩塌的问题。
